# class gymnasium.vector.VectorEnv

在`gymnasium`中，使用向量化环境的主要目的是提高计算效率，允许同时并行处理多个环境实例。这种方法特别适用于深度强化学习训练，其中算法可以从多个环境实例中同时收集经验，加快学习过程。以下是使用向量化环境的几个关键目的和好处：

### 1. **提高样本效率**
在强化学习中，算法需要大量的交互数据来学习策略。通过并行地从多个环境中收集数据，向量化环境可以显著增加每个时间步骤内可用的数据量，从而提高样本效率。

### 2. **加速训练过程**
并行处理多个环境实例可以更有效地利用现代计算资源（如多核CPU和GPU），加快训练过程。这对于计算密集型的强化学习算法尤其重要，它们通常需要大量的时间步骤来收敛到有效的策略。

### 3. **增强算法的鲁棒性**
通过在不同的环境实例中同时训练，强化学习算法可以在更多样化的状态分布上学习，这有助于提高学习到的策略的泛化能力和鲁棒性。

### 4. **简化实验设置**
向量化环境为研究人员和开发者提供了一种标准化的方法来同时管理和交互多个环境实例，简化了并行数据收集和实验设置的复杂性。

### 5. **提高数据收集的一致性**
在向量化环境中，所有子环境都可以被设计为同步或异步重置和执行步骤，这有助于确保数据收集过程的一致性和可控性，特别是当进行复杂实验设计或需要精确控制环境交互时。


`Gymnasium.vector.VectorEnv` 是 `gymnasium`库中的一个类，用于创建和管理多个环境实例，以便并行地执行和模拟。这种向量化的环境允许同时在多个副本上运行相同的操作，这对于加速训练过程特别有用，尤其是在强化学习领域。

### 为什么使用 `VectorEnv`?

在强化学习中，训练智能体通常需要大量的样本。使用 `VectorEnv` 可以并行地收集多个环境的样本，显著减少获取足够训练数据的时间。此外，它还可以提高资源的利用率，如CPU和GPU。

### `VectorEnv` 的工作原理

`VectorEnv` 基本上是一个容器，管理着多个环境实例。它提供了一组统一的API来在所有这些实例上执行操作，如`reset()`和`step()`，而不是单独对每个环境进行操作。当你对 `VectorEnv` 调用 `step` 方法时，它会在所有的环境上并行执行这个动作，并返回一个包含所有环境新状态的数组，以及相关的信息，如奖励和是否结束。

### 使用 `VectorEnv`

要使用 `VectorEnv`，你需要先创建一个或多个 `gymnasium` 环境，然后使用它们来初始化 `VectorEnv`。`gymnasium` 提供了几种不同的 `VectorEnv` 实现，比如 `SyncVectorEnv` 和 `AsyncVectorEnv`，分别用于同步和异步操作环境：

- **同步（Synchronous）**：在这种模式下，所有环境将按顺序执行，每个环境在移动到下一个之前必须完成当前步骤。
- **异步（Asynchronous）**：在这种模式下，所有环境可以同时独立执行，这可能会进一步提高效率，尤其是当环境的执行时间不一致时。

### 示例代码

下面是一个简单的示例，展示如何使用 `SyncVectorEnv` 来并行运行多个环境：

```python
import gymnasium as gym
from gymnasium.vector import SyncVectorEnv

def make_env(env_name):
    def _thunk():
        return gym.make(env_name)
    return _thunk

env_names = ["CartPole-v1" for _ in range(4)]  # 创建4个CartPole环境
envs = [make_env(name) for name in env_names]
vector_env = SyncVectorEnv(envs)

observations = vector_env.reset()
for _ in range(1000):
    actions = [vector_env.action_space.sample() for _ in range(vector_env.num_envs)]  # 随机选择动作
    observations, rewards, dones, infos = vector_env.step(actions)
    if all(dones):
        break

vector_env.close()
```

在这个例子中，我们创建了一个包含4个 `CartPole-v1` 环境的 `SyncVectorEnv`。然后，我们重置环境以获取初始观察结果，并在一个循环中执行随机动作，直到所有环境都完成。

`gymnasium.vector.VectorEnv` 是 Gymnasium 库中用于并行运行多个独立副本的同一环境的基类。向量化环境可以通过同时对多个子环境进行采样，以线性方式提高每秒步骤的执行速度。为了防止已终止的环境等待所有子环境终止或被截断，向量环境会在它们终止或被截断后自动重置子环境（在相同的步骤调用中）。结果是，步骤的观察结果和信息被重置的观察结果和信息覆盖。为了保留这些数据，最终步骤的观察结果和信息存储在信息参数中，使用“final_observation”和“final_info”分别存储。更多信息请参见`step()`方法。

向量环境为每个子环境批量处理观察结果、奖励、终止、截断和信息。此外，`step()`方法期望接收到每个并行环境的一批动作。

Gymnasium包含两种通用的向量环境：`AsyncVectorEnv` 和 `SyncVectorEnv`，以及几种自定义的向量环境实现。

### 使用向量环境的好处
- **并行性**: 通过并行执行多个环境，可以显著提高样本收集的速度，这对于需要大量数据的深度学习模型尤其重要。
- **效率**: 在多核处理器上，向量化环境能够充分利用硬件资源，提高计算效率。
- **简化API**: 向量化环境提供了一个统一的API来管理和交互多个环境，简化了代码的复杂性。

### 向量化环境的主要方法
- **reset()**: 重置所有子环境，并返回初始观察结果的批量。
- **step(actions)**: 对于每个子环境，执行一批动作，并返回观察结果、奖励、终止状态、截断状态和信息的批量。
- **close()**: 关闭所有子环境并释放资源。

### 示例代码
创建并使用向量化环境的示例代码可能如下所示：
```python
import gymnasium as gym
from gymnasium.vector import SyncVectorEnv

# 定义环境创建函数
def make_env(env_name):
    def _init():
        env = gym.make(env_name)
        return env
    return _init

env_names = ["CartPole-v1", "CartPole-v1"]  # 定义环境名称列表
env_fns = [make_env(name) for name in env_names]  # 创建环境函数列表

# 创建同步向量环境
vector_env = SyncVectorEnv(env_fns)

# 重置环境
observations = vector_env.reset()

# 在环境中执行动作
actions = [vector_env.action_space.sample() for _ in env_names]  # 为每个环境随机选择动作
observations, rewards, terminated, truncated, infos = vector_env.step(actions)

# 关闭环境
vector_env.close()
```
通过上述代码，可以在同一时间步骤内并行地执行两个`CartPole-v1`环境，从而加速数据收集过程。


### 向量环境的附加属性
- **num_envs**: 向量环境中子环境的数量。
- **observation_space**: 向量环境的批处理观察空间。
- **single_observation_space**: 单个子环境的观察空间。
- **action_space**: 向量环境的批处理动作空间。
- **single_action_space**: 单个子环境的动作空间。

这些属性帮助用户理解并行环境的内部结构和运作方式，尤其是批处理观察空间和动作空间的概念，这对于实现高效的并行数据采集和处理至关重要。

### 示例
以下示例展示了如何创建和使用向量化环境，并应用一些常见的包装器：

```python
import gymnasium as gym

# 创建包含3个同步的CartPole-v1环境的向量环境，并应用时间感知观察空间包装器
envs = gym.make_vec("CartPole-v1", num_envs=3, vectorization_mode="sync", wrappers=(gym.wrappers.TimeAwareObservation,))

# 应用奖励裁剪包装器，设置奖励的最小和最大值
envs = gym.wrappers.vector.ClipReward(envs, min_reward=0.2, max_reward=0.8)

# 重置环境并获取初始观察结果
observations, infos = envs.reset(seed=123)

# 执行动作并获取新的观察结果、奖励、终止和截断信号
_ = envs.action_space.seed(123)
observations, rewards, terminations, truncations, infos = envs.step(envs.action_space.sample())

# 关闭环境
envs.close()
```

### 注意事项
- **info参数的变化**：在v0.25之前，`reset()` 和 `step()` 的 info 参数实现为每个子环境的字典列表。从v0.25开始，这被修改为每个键为一个NumPy数组的字典。要使用旧的info样式，可以使用`DictInfoToList`包装器。
- **相同的观察和动作空间**：所有并行环境应共享相同的观察和动作空间，不支持多种不同环境的向量化。
- **`make_vec()`函数**：与`make()`函数类似，`make_vec()`专用于创建向量环境。

通过向量化环境，可以在多核处理器上并行执行多个环境副本，从而加速训练过程和数据采集，对于深度学习和强化学习算法的效率提升至关重要。

## Methods

`VectorEnv.step(actions: ActType)` 方法是向量环境中最重要的方法之一，它允许在每个并行环境中采取动作，并返回一批观察结果、奖励、终止信号、截断信号和额外信息。

### 参数
- **actions**: 动作的批处理数组，必须符合动作空间的形状。这意味着，你需要为向量环境中的每个子环境提供一个动作。

### 返回值
- **observations**: 新的观察结果的批处理数组。
- **rewards**: 奖励的数组，每个子环境一个奖励值。
- **terminations**: 终止信号的布尔数组，每个子环境一个信号，表示该子环境是否达到了终止状态。
- **truncations**: 截断信号的布尔数组，每个子环境一个信号，表示该子环境是否因为达到最大步数等条件而被截断。
- **infos**: 一个包含额外信息的字典，可能包括每个子环境特有的信息。

### 注意事项
由于向量环境会在子环境达到终止或截断状态时自动重置，返回的观察结果和信息并不是最后一步的观察结果或信息。相反，最后一步的观察结果和信息会被存储在`infos`字典中，通过键“final_observation”和“final_info”访问。

### 示例
以下示例展示了如何在包含3个`CartPole-v1`环境的向量环境中执行动作，并接收返回值：

```python
import gymnasium as gym
import numpy as np

# 创建包含3个同步的CartPole-v1环境的向量环境
envs = gym.make_vec("CartPole-v1", num_envs=3, vectorization_mode="sync")

# 重置环境并获取初始观察结果
_ = envs.reset(seed=42)

# 为每个子环境指定动作
actions = np.array([1, 0, 1], dtype=np.int32)

# 执行动作并获取返回值
observations, rewards, terminations, truncations, infos = envs.step(actions)

print("观察结果:", observations)
print("奖励:", rewards)
print("终止信号:", terminations)
print("截断信号:", truncations)
print("额外信息:", infos)
```

在这个示例中，我们为向量环境中的每个子环境指定了一个动作（1, 0, 1），并执行了这些动作。返回的观察结果、奖励、终止信号和截断信号都是批处理形式，即它们包含了所有子环境的相应值。这种批处理操作方式使得向量环境非常适合并行处理和加速训练过程。

`VectorEnv.reset()` 方法用于重置所有并行环境，并返回一批初始观察结果和额外信息（infos）。

### 参数
- **seed** (可选): 用于重置环境的种子，可以是单个整数、整数列表（为每个子环境指定不同的种子），或者`None`（不指定种子）。
- **options** (可选): 一个字典，用于向环境传递额外的重置选项。具体支持哪些选项取决于环境的实现。

### 返回值
- **观察结果**: 初始观察结果的批处理数组，数组的形状依赖于环境的观察空间。
- **infos**: 包含额外信息的字典，可能包括每个子环境特有的信息。

### 示例
以下示例展示了如何重置包含3个`CartPole-v1`环境的向量环境，并接收返回的初始观察结果和额外信息：

```python
import gymnasium as gym

# 创建包含3个同步的CartPole-v1环境的向量环境
envs = gym.make_vec("CartPole-v1", num_envs=3, vectorization_mode="sync")

# 重置环境，并指定种子为42
observations, infos = envs.reset(seed=42)

# 打印初始观察结果和额外信息
print("初始观察结果:", observations)
print("额外信息:", infos)
```

在这个示例中，我们通过`envs.reset(seed=42)`重置了所有的子环境，并为它们指定了相同的种子（42）。这使我们可以在每次运行时获得相同的初始状态，从而有助于实验的可重复性。返回的`observations`是一个包含每个子环境初始观察结果的数组，而`infos`字典包含了可能对进一步处理有用的额外信息。在此示例中，`infos`为空，这取决于具体环境是否在重置时提供了额外的信息。

`VectorEnv.render()` 方法用于从所有并行环境中获取渲染的帧。

### 返回值
- **渲染的帧**: 并行环境渲染的帧的元组。如果环境的`render_mode`不支持返回帧（例如，当`render_mode`设置为`"human"`时），则可能返回`None`。

### 示例
以下示例展示了如何创建一个向量环境，并渲染其帧。假设我们有一个支持`"rgb_array"`渲染模式的环境（例如`"CarRacing-v0"`），我们可以创建一个向量环境来并行运行多个实例，并渲染每个环境的帧。

```python
import gymnasium as gym

# 创建包含3个CarRacing-v0环境的向量环境，指定渲染模式为"rgb_array"
envs = gym.make_vec("CarRacing-v0", num_envs=3, vectorization_mode="sync", render_mode="rgb_array")

# 重置环境以开始新的回合
observations, infos = envs.reset()

# 执行一些动作
actions = [envs.single_action_space.sample() for _ in range(envs.num_envs)]
observations, rewards, terminations, truncations, infos = envs.step(actions)

# 渲染环境并获取渲染的帧
frames = envs.render()

# 检查和展示渲染的帧
if frames is not None:
    for i, frame in enumerate(frames):
        print(f"环境 {i+1} 的渲染帧尺寸:", frame.shape)
else:
    print("环境未返回渲染的帧")

# 关闭环境
envs.close()
```

在这个示例中，我们首先创建了一个包含3个`CarRacing-v0`环境实例的向量环境，并指定`render_mode`为`"rgb_array"`以允许环境返回渲染的帧。然后我们重置环境、执行一些随机动作，并通过`envs.render()`获取每个子环境的渲染帧。最后，我们检查返回的帧并打印出每个帧的尺寸，如果环境成功返回了渲染的帧的话。

请注意，不是所有环境都支持`"rgb_array"`渲染模式，所以在实际使用时需要确保所使用的环境支持所需的渲染模式。此外，如果环境配置为`"human"`渲染模式，`envs.render()`可能会返回`None`，因为`"human"`模式下渲染的目的是直接在屏幕上显示，而不是返回帧数据。

`VectorEnv.close()` 方法用于关闭所有的并行环境，并释放相关资源。这个方法是向量环境中非常重要的一个环节，确保在环境不再被需要时，可以适当地清理和释放资源。

### 功能
- **关闭所有并行环境**: 关闭并行运行的所有子环境，结束每个环境的运行并释放占用的资源。
- **关闭图像查看器**: 如果有任何图像查看器（如在渲染时创建的）被打开，此方法会负责关闭它们。
- **调用`close_extras()`**: 执行额外的清理工作，这个方法应该在子类中实现具体的环境关闭逻辑。
- **设置`closed`状态为True**: 标记这个向量环境为已关闭，防止后续再被错误地使用。

### 注意
- 虽然`close()`方法负责启动清理过程，但实际上关闭环境的具体动作应该在`close_extras()`方法中实现。这意味着，如果你正在开发自定义的向量环境，你需要确保在`close_extras()`方法中适当地关闭每个子环境。
- 当垃圾收集器回收向量环境对象时，或者程序退出时，`close()`方法会自动被调用。这是为了确保即使在异常情况下，资源也能被正确释放。

### 示例代码
以下代码演示了如何创建一个向量环境，执行一些动作，然后关闭环境：

```python
import gymnasium as gym

# 创建一个向量环境，包含3个CartPole-v1子环境
envs = gym.make_vec("CartPole-v1", num_envs=3, vectorization_mode="sync")

# 重置环境，开始新的回合
observations, infos = envs.reset()

# 执行一些动作
for _ in range(10):
    actions = [envs.single_action_space.sample() for _ in range(envs.num_envs)]
    observations, rewards, terminations, truncations, infos = envs.step(actions)

# 关闭环境，释放资源
envs.close()
```

在这个例子中，我们首先创建了一个包含3个`CartPole-v1`子环境的向量环境，然后执行了10次随机动作。最后，我们调用`close()`方法来关闭环境并释放资源。

## Attributes

`VectorEnv.num_envs` 属性表示向量环境中子环境的数量。这是一个整数值，指明了并行运行的环境数量。这个属性对于理解整个向量环境的规模很重要，也可以用来循环执行特定于每个环境的操作。

### 功能
- **获取子环境数量**: 提供了一个简单的方式来了解并行环境的数量，这对于批量处理观测结果、动作和奖励非常有用。

### 示例代码
以下示例代码展示了如何获取并使用`num_envs`属性：

```python
import gymnasium as gym

# 创建一个向量环境，包含3个CartPole-v1子环境
envs = gym.make_vec("CartPole-v1", num_envs=3, vectorization_mode="sync")

# 获取子环境的数量
num_envs = envs.num_envs
print(f"Number of sub-environments: {num_envs}")

# 重置环境，开始新的回合
observations, infos = envs.reset()

# 对每个子环境执行随机动作
for _ in range(10):
    actions = [envs.single_action_space.sample() for _ in range(num_envs)]
    observations, rewards, terminations, truncations, infos = envs.step(actions)

# 关闭环境，释放资源
envs.close()
```

在这个例子中，`num_envs`属性被用来确定向量环境中有多少个子环境，这样我们就可以为每个子环境生成相应数量的随机动作。这对于批量处理操作非常有用，确保了我们能够正确地与向量环境中的所有子环境进行交互。

`VectorEnv.action_space` 属性定义了向量环境中所有子环境共享的（批量化的）动作空间。这意味着，当你向环境的`step`方法提供动作时，这些动作必须是`action_space`中有效元素的集合。

### 功能
- **动作的验证**: 确保所有发送给环境的动作都是有效的，且符合环境期望的数据结构和类型。
- **动作空间的探索**: 了解可用动作的范围，包括动作的类型（离散或连续）、维度大小等。

### 示例代码
以下示例代码展示了如何获取并使用`action_space`属性：

```python
import gymnasium as gym
import numpy as np

# 创建一个向量环境，包含3个子环境
envs = gym.make_vec("CartPole-v1", num_envs=3, vectorization_mode="sync")

# 获取批量化的动作空间
action_space = envs.action_space
print(f"Batched action space: {action_space}")

# 生成一个批量动作的示例
actions = action_space.sample()
print(f"Sample batch of actions: {actions}")

# 使用批量动作执行一步
observations, rewards, terminations, truncations, infos = envs.step(actions)

# 关闭环境，释放资源
envs.close()
```

在这个例子中，我们首先获取向量环境的批量化动作空间，然后使用`action_space.sample()`方法生成一个有效的批量动作示例。这个批量动作然后被用于`step`方法，以在环境中执行动作。这种方法允许在多个子环境中同时执行动作，从而提高了样本效率和执行速度。

`VectorEnv.observation_space` 属性定义了向量环境中所有子环境共享的（批量化的）观察空间。这意味着，由`reset`和`step`方法返回的观察结果必须是`observation_space`中有效元素的集合。

### 功能
- **观察结果的验证**: 确保所有从环境返回的观察结果都是有效的，并且符合环境期望的数据结构和类型。
- **观察空间的探索**: 了解环境可能产生的观察结果的范围，包括观察结果的类型（如图像、向量等）、维度大小等。

### 示例代码
以下示例代码展示了如何获取并使用`observation_space`属性：

```python
import gymnasium as gym

# 创建一个向量环境，包含3个子环境
envs = gym.make_vec("CartPole-v1", num_envs=3, vectorization_mode="sync")

# 获取批量化的观察空间
observation_space = envs.observation_space
print(f"Batched observation space: {observation_space}")

# 重置环境并获取初始观察结果
observations, infos = envs.reset(seed=42)
print(f"Initial batch of observations: {observations}")

# 关闭环境，释放资源
envs.close()
```

在这个例子中，我们首先获取向量环境的批量化观察空间，然后通过调用`reset`方法来重置环境并获取初始的批量观察结果。这个批量观察结果是`observation_space`的有效元素，它为后续的决策提供了环境状态的信息。通过这种方式，向量环境支持在多个子环境中并行生成观察结果，从而提高了样本效率和执行速度。

`VectorEnv.single_action_space` 属性定义了向量环境中单个子环境的动作空间。这意味着在向量环境中的每个子环境上执行的动作必须是`single_action_space`中的有效元素。

### 功能
- **动作验证**: 确保向每个子环境发送的动作都是有效的，并且符合预期的数据结构和类型。
- **动作空间探索**: 了解单个环境接受的动作的范围，包括动作的类型（如离散动作、连续动作等）、大小等。

### 示例代码
以下示例代码展示了如何获取并使用`single_action_space`属性：

```python
import gymnasium as gym

# 创建一个向量环境，包含3个子环境
envs = gym.make_vec("CartPole-v1", num_envs=3, vectorization_mode="sync")

# 获取单个子环境的动作空间
single_action_space = envs.single_action_space
print(f"Single action space: {single_action_space}")

# 生成一个有效的动作样本
single_action = single_action_space.sample()
print(f"Sample action for a single environment: {single_action}")

# 关闭环境，释放资源
envs.close()
```

在这个例子中，`single_action_space`属性被用来获取单个子环境的动作空间，并通过调用`sample`方法生成一个有效的动作样本。这对于理解和探索环境接受的动作类型非常有用。即便在一个向量化的环境中，这些动作空间通常是相同的，理解单个动作空间的属性对于设计适当的策略和动作转换非常重要。

`VectorEnv.single_observation_space` 属性定义了向量环境中单个子环境的观察空间。这意味着在向量环境中的每个子环境上产生的观察值都必须是`single_observation_space`中的有效元素。

### 功能
- **观察验证**: 确保从每个子环境返回的观察值都是有效的，并且符合预期的数据结构和类型。
- **观察空间探索**: 了解单个环境产生的观察的范围，包括观察的类型（如图像、向量等）、大小和数据类型等。

### 示例代码
以下示例代码展示了如何获取并使用`single_observation_space`属性：

```python
import gymnasium as gym

# 创建一个向量环境，包含3个子环境
envs = gym.make_vec("CartPole-v1", num_envs=3, vectorization_mode="sync")

# 获取单个子环境的观察空间
single_observation_space = envs.single_observation_space
print(f"Single observation space: {single_observation_space}")

# 重置环境并获取初始观察值
observations, _ = envs.reset()
# 显示第一个子环境的初始观察值的形状
print(f"Shape of initial observation from a single environment: {observations[0].shape}")

# 关闭环境，释放资源
envs.close()
```

在这个例子中，`single_observation_space`属性被用来获取单个子环境的观察空间，并通过调用`reset`方法并查看返回的观察值来探索这个空间。这对于理解环境中可能出现的观察值的类型和结构非常有用，尤其是在设计处理这些观察值的模型和算法时。即便在一个向量化的环境中，这些观察空间通常是相同的，理解单个观察空间的属性对于设计有效的观察处理策略至关重要。

`VectorEnv.spec` 属性提供了环境的规范，通常在通过 `gymnasium.make_vec()` 函数创建向量环境时设置。这个规范包括了环境的唯一标识符、版本号、奖励范围、最大步数等重要信息，有助于理解和管理环境的行为。

### 功能
- **环境信息**: 提供关于环境的详细信息，如环境的ID、观察空间和动作空间的类型、每个回合的最大步数等。
- **环境管理**: 通过`spec`属性，可以方便地管理和查询环境配置，尤其是在使用多个环境或在不同上下文中重用环境时。

### 示例代码
以下示例代码展示了如何访问和使用`VectorEnv.spec`属性：

```python
import gymnasium as gym

# 创建一个向量化环境
envs = gym.make_vec("CartPole-v1", num_envs=3, vectorization_mode="sync")

# 获取环境规范
env_spec = envs.spec
print(f"Environment ID: {env_spec.id}")
print(f"Max episode steps: {env_spec.max_episode_steps}")
print(f"Reward range: {env_spec.reward_range}")

# 关闭环境
envs.close()
```

在这个例子中，`spec`属性被用来获取向量环境的规范信息。通过打印`spec`的不同属性，我们可以了解环境的基本配置，如环境的ID、每个回合的最大步数以及奖励的范围。这对于评估和调整训练策略非常有用，特别是在处理多个环境或需要对环境进行细粒度控制时。

`VectorEnv.render_mode` 属性指定了向量环境的渲染模式，这个模式定义了环境如何和应该如何渲染。这个属性遵循和单个环境中`Env.render_mode`相似的规范。向量环境的渲染模式对于可视化和调试多个环境实例同时运行的情况非常有用。

### 功能
- **渲染控制**: 控制环境的可视化展示，比如是否显示环境的图形界面。
- **兼容性**: 确保向量环境的渲染模式与单个环境的渲染模式兼容，便于统一处理。

### 支持的渲染模式
- `None`: 不进行任何渲染。
- `"human"`: 以人类可视的形式渲染环境，通常是图形窗口。
- `"rgb_array"`: 以RGB数组的形式返回环境的视图，适用于进一步处理或网络输入。
- 可能还有其他自定义渲染模式，具体取决于环境。

### 示例代码
以下是一个如何设置和使用`VectorEnv.render_mode`属性的示例：

```python
import gymnasium as gym

# 创建一个向量化环境，指定渲染模式为'rgb_array'
envs = gym.make_vec("CartPole-v1", num_envs=3, vectorization_mode="sync", render_mode="rgb_array")

# 进行一步动作，观察渲染输出
_ = envs.reset(seed=42)
observations, _, _, _, infos = envs.step([0, 1, 0])

# 如果环境支持'rgb_array'模式，渲染的图像会在infos字典中以'rgb_array'键返回
if "rgb_array" in infos:
    images = infos["rgb_array"]
    print("Rendered images shape:", images.shape)

# 关闭环境
envs.close()
```

在这个示例中，`render_mode`被设置为`"rgb_array"`，这意味着环境在每次`step`调用时都会返回环境视图的RGB数组。这些数组可以用于视频录制、实时展示或作为深度学习模型的输入。

`VectorEnv.closed` 属性表示向量环境是否已经被关闭。这是一个布尔值，用于指示环境的当前状态。

### 功能
- **状态指示**: 指示向量环境是否已经通过调用`close()`方法被关闭，防止在已关闭的环境上执行操作，这可能导致错误或异常。
- **资源管理**: 正确管理环境资源，如图形界面、内存等，确保在环境不再需要时释放这些资源。

### 示例代码
以下示例展示了如何检查向量环境的`closed`状态，并基于该状态决定是否执行某些操作：

```python
import gymnasium as gym

# 创建向量环境
envs = gym.make_vec("CartPole-v1", num_envs=3, vectorization_mode="sync")

# 检查环境是否已关闭
if not envs.closed:
    # 如果环境未关闭，执行一些操作
    _ = envs.reset(seed=42)
    _, _, _, _, _ = envs.step([1, 0, 1])
    envs.close()  # 关闭环境

# 再次检查环境是否已关闭
if envs.closed:
    print("环境已经成功关闭。")
else:
    print("环境尚未关闭。")
```

在这个示例中，我们首先检查`envs.closed`的值来确定环境是否已经被关闭。如果环境未关闭，我们继续执行一些步骤动作并最终关闭环境。关闭环境后，我们再次检查`closed`属性来验证环境是否确实已经被关闭。

关闭环境是一个重要的步骤，特别是在使用图形界面或需要释放大量资源的环境时。正确地管理环境的打开和关闭状态可以帮助确保资源被妥善管理，避免内存泄漏等问题。

## Additional Methods

`VectorEnv.unwrapped` 属性提供了一种访问向量环境中基础环境的方法。这对于需要直接与底层环境进行交互的情况非常有用，例如，当你需要调用基础环境特有的方法或访问特定属性时。

### 功能
- **直接访问**: 允许用户绕过所有包装器，直接访问向量化环境中的基础环境。
- **调用特定操作**: 对于一些特定操作或属性访问，只存在于基础环境中而不是向量环境或其包装器中，`unwrapped`属性提供了直接调用这些操作的能力。

### 示例代码
以下示例假设我们有一个自定义环境，该环境包含一些特定的方法或属性，这些方法或属性在向量环境或其任何包装器中都不可用。我们将展示如何使用`unwrapped`属性访问这些方法或属性：

```python
import gymnasium as gym

# 假设 CustomEnv 是一个包含特定方法或属性的自定义环境
# envs = gym.make_vec("CustomEnv-v0", num_envs=3, vectorization_mode="sync")

# 使用 unwrapped 属性访问基础环境
base_env = envs.unwrapped

# 现在你可以直接调用基础环境的特定方法或访问其属性
# base_env.custom_method()
# print(base_env.custom_attribute)
```

在这个示例中，我们首先通过`make_vec`方法创建了一个向量环境`envs`。然后，我们使用`unwrapped`属性来获取对基础环境的直接访问。一旦获得基础环境的引用，我们就可以自由地调用在向量化环境中不可用的任何特定方法或访问任何特定属性。

### 注意事项
- 在使用`unwrapped`属性时，需要了解返回的基础环境可能不支持向量操作，即在该环境上执行的操作将不会自动应用于所有子环境。
- `unwrapped`属性的行为可能会根据具体实现的不同而有所差异，特别是当环境通过多层包装器封装时。在某些情况下，`unwrapped`可能返回最底层的环境实例，而在其他情况下，它可能返回某个中间层次的包装环境。

`VectorEnv.np_random` 属性提供了对向量环境内部使用的伪随机数生成器（PRNG）的访问。这允许用户获取或设置环境内部用于生成随机数的`numpy.random.Generator`实例，从而可以进行更细致的随机性控制，例如重置种子以实现结果的可复现性。

### 功能
- **随机性控制**: 通过访问和设置`np.random.Generator`实例，用户可以控制环境中的随机性，例如，通过重置种子。
- **初始化**: 如果`_np_random`没有被显式设置，访问`np_random`属性会自动初始化一个新的`Generator`实例，使用随机种子。

### 示例代码
以下示例演示了如何访问和使用`VectorEnv`的`np_random`属性来重置环境的随机种子：

```python
import gymnasium as gym
import numpy as np

# 创建向量环境实例
envs = gym.make_vec("CartPole-v1", num_envs=3, vectorization_mode="sync")

# 获取环境的随机数生成器
random_gen = envs.np_random

# 使用特定种子重置随机数生成器
seed = 42
random_gen.seed(seed)

# 验证种子重置
# 这将确保环境的随机性操作（如随机选择动作或环境初始化）可以被可复现地重现
# 示例：重置环境并采取随机动作
observations, infos = envs.reset(seed=seed)
actions = [envs.single_action_space.sample() for _ in range(envs.num_envs)]
observations, rewards, terminations, truncations, infos = envs.step(actions)

# 打印一些结果以验证随机性控制
print(observations)
```

在这个示例中，我们首先创建了一个向量环境`envs`。然后，我们通过`np_random`属性访问环境的随机数生成器，并使用特定种子重置它。这样做可以确保环境内部所有基于随机性的操作都将基于这个新种子进行，从而提高实验的可复现性。

### 注意事项
- 对`np_random`的修改会影响环境内所有随机性操作的结果，包括但不限于动作的选择、环境的初始状态以及任何内部随机决策。
- 在并行环境中，`np_random`通常用于管理所有子环境的随机性。因此，改变它可能会同时影响到所有并行执行的环境。